# FEM-BEM coupling 

In [ ]:
from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw
from time import time

In [ ]:
ball = Sphere(Pnt(0,0,0), 1).bc("fembem")
elec = Cylinder(Pnt(-0.5,0,0), Pnt(0.5,0,0),0.6).maxh(0.05) * \
    (OrthoBrick(Pnt(-0.3,-1,-1), Pnt(-0.2,1,1))+OrthoBrick(Pnt(0.2,-1,-1), Pnt(0.3,1,1)))
elec.bc("electrode")
geo = CSGeometry()
geo.Add(ball-elec)
mesh = Mesh(geo.GenerateMesh(maxh=0.25))
mesh.Curve(2)
Draw (mesh, clipping={"pnt":(0,0,0)})

second order space for FEM, but only first order space for BEM:

In [ ]:
V = H1(mesh,order=2, dirichlet="electrode")
Vt = H1(mesh, order=1, definedon=mesh.Boundaries("fembem"))
Q = SurfaceL2(mesh, order=0, definedon=mesh.Boundaries("fembem"))
X = V*Vt*Q
(u,ut,lam), (v,vt,mu) = X.TnT()

In [ ]:
f = LinearForm(X)
# f += v(0.7,0,0)  # point source
f.Assemble()

dsfb = ds(definedon="fembem")
a = BilinearForm(grad(u)*grad(v)*dx - lam*v*dsfb + (ut-u)*vt*dsfb).Assemble()
b = BilinearForm(grad(u)*grad(v)*dx+u*v*dx + ut*vt*dsfb + lam*mu*dsfb).Assemble()
inv = b.mat.Inverse(inverse="sparsecholesky", freedofs=X.FreeDofs())

gf = GridFunction(X)
gfu, gfut, gflam = gf.components
gfu.Set(IfPos(x, 1, -1), BND)  # set Dirichlet bnd values

In [ ]:
# Draw (gfu, clipping={ "pnt" : (0,0,0) })

Next, we assemble the corresponding boundary integral operators using Bempp-cl and ngbem

In [ ]:
import ngbem
import bempp.api;
import numpy as np;
import scipy;
import bempp.core

# bempp.api.DEFAULT_DEVICE_INTERFACE="opencl"
bempp.api.DEFAULT_DEVICE_INTERFACE="numba"

NGBem implements the trace operator between (in our case) $H^1(\Omega)$ and $H^{1/2}(\partial \Omega)$. It
also provides the mapping between the NGSolve SurfaceL2 and the Bempp-cl space of piecewise constant functions.
In order to make sure that the grids and spaces match, H1_trace also returns a corresponding Bempp-cl function space. (The second parameter makes sure that we only generate the Bempp grid once).

Note that, since we are using piecewise constant functions for $Q$, bnd_trace_matrix will be the identity

In [ ]:
fembem = mesh.Boundaries("fembem")
grid = ngbem.bempp_grid_from_ng(mesh, fembem)
bem_c, trace_matrix = ngbem.ng_to_bempp_trace(Vt, grid, fembem);
bem_dc, bnd_trace_matrix = ngbem.ng_to_bempp_trace(Q,grid, fembem);

# We will need the following operators:
the single layer operator $V$,
the double layer operator $K$ and a mass matrix $M$

In [ ]:
##set up the bem
bempp.api.VECTORIZATION_MODE = "novec" 
laplace = bempp.api.operators.boundary.laplace
sl=laplace.single_layer(bem_dc,bem_c,bem_dc) # ,assembler="fmm", device_interface="opencl")
dl=laplace.double_layer(bem_c,bem_c,bem_dc)  #,assembler="fmm", device_interface="opencl")

identity = bempp.api.operators.boundary.sparse.identity
id_op=identity(bem_dc,bem_dc,bem_c)
id_op2=identity(bem_c,bem_c,bem_dc)

In [ ]:
embu, embut, emblam = X.embeddings

In [ ]:
bnd_op1=0.5*id_op2 - dl
ngs_bnd1= BaseMatrix(bnd_trace_matrix.T) @ BaseMatrix(bnd_op1.weak_form()) @ BaseMatrix(trace_matrix)
ngs_bnd1= emblam @ ngs_bnd1 @ embut.T

We set up the following block system:
$\begin{pmatrix}
A & M \\
\frac{1}{2} M - K & V
\end{pmatrix} 
\begin{pmatrix}
u \\ \lambda
\end{pmatrix}= \begin{pmatrix} f \\ 0 \end{pmatrix}$

In [ ]:
ngs_sl = BaseMatrix(bnd_trace_matrix.T) @ BaseMatrix(sl.weak_form()) @ BaseMatrix(bnd_trace_matrix) 
ngs_sl = emblam @ ngs_sl @ emblam.T   # 1,1 block

In [ ]:
lhs=a.mat + ngs_sl + ngs_bnd1

In [ ]:
solvers.GMRes(A=lhs, b=f.vec, pre=inv, tol=1e-6, printrates=True, x=gf.vec,freedofs=X.FreeDofs(),maxsteps=400)
pass

In [ ]:
gfu, gfut, gflam = gf.components
Draw(gfu, clipping = { "pnt" : (0,0,0) })
Draw(gflam, draw_vol=False)